<a href="https://colab.research.google.com/github/bekimpilo/2024-openai-gpt-hiring-racial-discrimination/blob/main/stockouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarizing Google News Headlines Using VertexAI PALM API & Langchain

In this tutorial, we will perform news summarization of news content gathered from Google News using the following components:

- GNews API: Collect news titles & metadata from Google News
- Langchain's UnstructuredURLLoader: Retrieve news content
- Vertex PALM API: Generate news summary

Vertex PALM API is a large language model (LLM) that can be used for a variety of tasks, including text summarization. In this tutorial, we will use the text-bison@001 model from PALM API to summarize news content.

Reference and credit to the following resources:
- https://github.com/ranahaani/GNews
- https://alphasec.io/summarize-google-news-results-with-langchain-and-serper-api/

## Objectives:
- Learn how to use GNews API, Langchain's UnstructuredURLLoader, and Vertex PALM API to perform news summarization
- Create a news summarization function that can be used to automate the process of generating news summaries
- Gain a better understanding of the different steps involved in news summarization

## Installation & Preparation

In [ ]:
#install all required package
!pip -q install langchain
!pip install google-cloud-aiplatform
!pip install gnews
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=9632bf328222ece93803d10396be75bea0af7e305616084626611dfd86507906
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
# restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# import required packages
from langchain.llms import VertexAI
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import UnstructuredURLLoader


ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
# authenticate to google cloud account
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
#google cloud project name
#replace with your project name

import vertexai

PROJECT_ID = "llms-419612"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

## Calling GNews API to Get News Metadata
Limit news period using the following time operators:
 - h = hours (eg: 12h)
 - d = days (eg: 7d)
 - m = months (eg: 6m)
 - y = years (eg: 1y)

Example:

`google_news.period = '3d'  # News from last 3 days `

In [ ]:
from gnews import GNews

google_news = GNews()
google_news.period = '1y'  # News from last 1 day
google_news.max_results = 100  # number of responses across a keyword
#google_news.country = 'ID'  # News from a specific country = Indonesia
google_news.language = 'en'  # News in a specific language = Bahasa Indonesia
#google_news.exclude_websites = ['yahoo.com', 'cnn.com', 'msn.con']  # Exclude news from specific website i.e Yahoo.com and msn.com

#use date range if required
google_news.start_date = (2024, 1, 1) # Search from 1st Jan 2023
google_news.end_date = (2024, 12, 1) # Search until 1st April 2023

#get by keyword
news_by_keyword = google_news.get_news('recalls')

In [ ]:
#check collected news metadata
news_by_keyword

## Extract news content
The `UnstructuredURLLoader` from Langchain library is usefull toolkit to get easy access to HTML contents from a url. This package is actually a wrapper of `bricks.html` partition from [Unstructured](https://unstructured-io.github.io/unstructured/bricks/partition.html#partition-html) library.  We will use it as a news content extractor by taking input from url collected at previous steps.

In [ ]:
#prompting to perform news summary
prompt_template = """Generate summary for the following text, using the following steps:
                     1. Perform named entity recognition identify products, organisations, country
                     2. If the text cannot be found or error, return: "Content empty"
                     3. Use only materials from the text supplied
                     4. Create in English

                    "{text}"
                    SUMMARY:"""

prompt = PromptTemplate.from_template(prompt_template)

#declare LLM model
llm = VertexAI(temperature=0.1,
               model='gemini-1.0-pro',
               top_k=40,
               top_p=0.8,
               max_output_token=512)

Wrap the summarization process inside a function to loop collections of news urls. The generate_summary function perform the following:
- Retrieve news content from each urls
- Generate summary for each news contents
- Print the output

In [ ]:
# create function to generate news summary based on list of news urls
# Load URL , get news content and summarize
def generate_summary(docnews):
    for item in docnews:
        #extract news content
        loader = UnstructuredURLLoader(urls=[item['url']])
        data = loader.load()

        #summarize using stuff for easy processing
        chain = load_summarize_chain(llm,
                                    chain_type="stuff",
                                    prompt=prompt)
        summary = chain.run(data)

        #show summary for each news headlines
        print(item['title'])
        print(item['publisher']['title'], item['published date'])
        print(summary, '\n')

In [ ]:
#call the function and generate summary for news by keyword
generate_summary(news_by_keyword)

https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-gemini-function-calling?hl=en